In [ ]:
import os
import numpy as np
import TissueParameterList as TPL
import nibabel as nib 

from pathlib import Path
root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"

fpath_input = root_path / "Input/"

fname_xml = fpath_input / "XCAT_TissueParameters_XML.xml"
tpl = TPL.TissueParameterList()
tpl.parse_xml(str(fname_xml))
label_name_dict = tpl.get_labels_and_names()


In [ ]:
from scipy.ndimage.morphology import binary_erosion
import matplotlib.pyplot as plt
def get_mask_for_labels(segmentation, labels):
    mask = np.zeros(segmentation.shape, dtype=bool)
    for lab in labels:
        mask += (segmentation == lab)
    
    plt.imshow(mask)
#     mask = binary_erosion(mask)
    return mask

def get_myocard_mask(segmentation):
    labels_myocard = [1,2,3,4]
    return get_mask_for_labels(segmentation, labels_myocard)


def load_data(fname):
    
    data = np.load(fname)
    
    labels= data["labels"]
    gt_t1 = data["gt_t1"]
    gt_t2 = data["gt_t2"]
    match_t1 = data["match_t1"]
    match_t2 = data["match_t2"]
    
    mask = get_myocard_mask(labels)
    
    return gt_t1, gt_t2, match_t1, match_t2, mask, labels
    

def analyse_myocardium(t1map, t2map, mask):
    
    t1 = t1map[mask]
    t2 = t2map[mask]
    
    return t1, t2


In [ ]:
def analyse_dataset(fpath_data):
    print("Loading {}".format(fpath_data))

    gt_t1, gt_t2, match_t1, match_t2, mask, labels = load_data(fpath_data)

    f,ax = plt.subplots(2,2)
    ax[0,0].imshow(gt_t1,cmap='jet', vmin=0, vmax=2500)
    ax[1,0].imshow(gt_t2,cmap='magma', vmin=10, vmax=150)
    ax[0,1].imshow(match_t1,cmap='jet', vmin=0, vmax=2500)
    ax[1,1].imshow(match_t2,cmap='magma', vmin=10, vmax=150)
    
    t1_gt_val, t2_gt_val = analyse_myocardium(gt_t1, gt_t2, mask)
    match_t1_val, match_t2_val = analyse_myocardium(match_t1, match_t2, mask)
    return t1_gt_val, t2_gt_val, match_t1_val, match_t2_val


In [ ]:
import numpy as np

parameters = ["T1", "T2"]
fittype = ["GT", "dictionary"]
motion = ["stat", "resp", "card", "cardioresp", "half resp", "card gated"]
valuetype = ["mean", "std"]

mean_values = np.zeros(shape=(len(parameters),len(fittype),len(motion)), dtype=object)
std_values = np.zeros(shape=(len(parameters),len(fittype),len(motion)), dtype=object)


In [ ]:
fpath_data = root_path / "Output/output_i_fitresults_stat.npz"
res = analyse_dataset(fpath_data)

values[0,0,0,0] = np.mean(res[0])
values[1,0,0,0] = np.mean(res[1])
values[0,1,0,0] = np.mean(res[2])
values[1,1,0,0] = np.mean(res[3])

values[0,0,0,1] = np.std(res[0])
values[1,0,0,1] = np.std(res[1])
values[0,1,0,1] = np.std(res[2])
values[1,1,0,1] = np.std(res[3])

print(values)

In [ ]:
import xarray as xr

data = xr.DataArray(data=values, coords=[parameters,fittype,motion], dims=["parameter", "fittype", "motion"])
print(data)

In [ ]:
import matplotlib.pyplot as plt
print(data.sel(parameter="T1", fittype="dictionary"))